# RL Policy Training - SAC

Train SAC (Soft Actor-Critic) with continuous action space to learn service policies.

Key differences from DQN:
- Continuous action space: agent outputs delay directly [0, max_delay]
- Off-policy (like DQN) - efficient replay buffer usage
- Entropy regularisation - helps exploration in noisy environments

See also:
- `rl_dqn.ipynb` - DQN with discrete action space

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from src.rl import (
    ServiceEnv,
    get_default_scenario,
    print_scenario_info,
    compare_with_baselines,
    run_sanity_check,
    evaluate_model,
    RewardLoggerCallback,
    EpisodeDiagnosticsCallback,
    ContinuousActionStatsCallback,
    DEFAULT_MAX_TIME,
)

## 1. Setup Scenario

In [ ]:
# Use shared scenario setup
scenario = get_default_scenario()
print_scenario_info(scenario)

MAX_TIME = DEFAULT_MAX_TIME

## 2. Baseline Comparison

In [ ]:
# Compare baseline policies
results = compare_with_baselines(scenario, max_time=MAX_TIME)

In [ ]:
# Sanity check: Run optimal policy through RL environment
sanity_results = run_sanity_check(scenario, max_time=MAX_TIME, fixed_durability=1.0)
print(f"\nExpected: Should match 'optimised_linear' baseline (~{results['optimised_linear']['mean']:.2f})")

## 3. Train SAC

SAC uses continuous action space - the agent outputs the delay directly.

In [ ]:
# Train SAC
import torch.nn as nn
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback

# Environment settings
MAX_ACTION_DELAY = 100.0  # Maximum delay agent can output
REWARD_SCALE = None       # None = use failure_cost (rewards in ~[0, 2.5])
FIXED_DURABILITY = 1.0    # Fix durability for debugging (None = use generated)

# SAC hyperparameters
LEARNING_RATE = 3e-4
BATCH_SIZE = 256          # SAC typically uses smaller batches than DQN needed
GAMMA = 0.99
TAU = 0.005               # Soft update coefficient (SAC default)
BUFFER_SIZE = 50_000

# Entropy coefficient (auto-tuned by default)
ENT_COEF = 'auto'         # Or set to fixed value like 0.1

# Checkpointing
SAVE_CHECKPOINTS = True
CHECKPOINT_FREQ = 10_000

# Create training environment with CONTINUOUS action space
env = ServiceEnv(
    scenario, 
    max_time=MAX_TIME, 
    seed=42,
    reward_scale=REWARD_SCALE,
    max_action_delay=MAX_ACTION_DELAY,
    fixed_durability=FIXED_DURABILITY,
    continuous_actions=True,  # <-- Key difference from DQN
)

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")
print(f"Reward scale: {env.reward_scale}")
print(f"Fixed durability: {FIXED_DURABILITY}")

# Optimal interval for durability=1.0: 24.2 + 24.2*1.0 = 48.4
if FIXED_DURABILITY is not None:
    opt_interval = 24.2 + 24.2 * FIXED_DURABILITY
    print(f"Optimal interval (from linear policy): {opt_interval:.1f}")

# Create evaluation environments
eval_env = ServiceEnv(
    scenario, max_time=MAX_TIME, seed=123,
    max_action_delay=MAX_ACTION_DELAY, fixed_durability=FIXED_DURABILITY,
    continuous_actions=True,
)
optimal_env = ServiceEnv(
    scenario, max_time=MAX_TIME, seed=123,
    max_action_delay=MAX_ACTION_DELAY, fixed_durability=FIXED_DURABILITY,
    continuous_actions=True,
    use_optimal_policy=True,
)

# Wrap training env for logging
env = Monitor(env)

# Network architecture
policy_kwargs = dict(
    net_arch=[64, 64],
    activation_fn=nn.ReLU,
)

# Print configuration
print(f"\nSAC Configuration:")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Soft update tau: {TAU}")
print(f"  Entropy coef: {ENT_COEF}")
print(f"  Network: {policy_kwargs}")

# TensorBoard: run `tensorboard --logdir ./logs` to watch
print("\nTraining SAC (verbose=0, use TensorBoard to monitor)...")

model = SAC(
    'MlpPolicy',
    env,
    learning_rate=LEARNING_RATE,
    buffer_size=BUFFER_SIZE,
    batch_size=BATCH_SIZE,
    gamma=GAMMA,
    tau=TAU,
    ent_coef=ENT_COEF,
    policy_kwargs=policy_kwargs,
    verbose=0,
    seed=42,
    tensorboard_log="./logs",
)

# Callbacks for logging
reward_logger = RewardLoggerCallback()
callbacks = [
    reward_logger,
    ContinuousActionStatsCallback(log_freq=1000),  # For continuous actions
    EpisodeDiagnosticsCallback(
        max_time=MAX_TIME,
        log_freq=100,
        eval_env=eval_env,
        optimal_env=optimal_env,
        n_eval_episodes=50,
    ),
]

# Optional: Save checkpoints during training
if SAVE_CHECKPOINTS:
    checkpoint_callback = CheckpointCallback(
        save_freq=CHECKPOINT_FREQ,
        save_path='./checkpoints/',
        name_prefix='sac_service',
    )
    callbacks.append(checkpoint_callback)
    print(f"  Checkpoints: every {CHECKPOINT_FREQ} steps to ./checkpoints/")

model.learn(total_timesteps=200_000, callback=callbacks)

# Store for later cells
training_rewards = reward_logger.episode_rewards
trained_model = model

print(f"\nTraining complete!")
print(f"Episodes: {len(training_rewards)}")

In [ ]:
# Plot training progress
fig, ax = plt.subplots(figsize=(10, 4))

rewards = training_rewards
window = 50
if len(rewards) > window:
    smoothed = np.convolve(rewards, np.ones(window)/window, mode='valid')
    ax.plot(smoothed, label=f'Rolling avg (window={window})')
    ax.scatter(range(len(rewards)), rewards, alpha=0.1, s=1, label='Raw')
else:
    ax.plot(rewards)

ax.set_xlabel('Episode')
ax.set_ylabel('Cumulative Reward')
ax.set_title('SAC Training Progress')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Evaluate Trained Model

In [ ]:
# Evaluate trained SAC model
sac_rewards = evaluate_model(
    trained_model,
    scenario,
    n_episodes=500,
    max_time=MAX_TIME,
    seed=42,
    deterministic=True,
    max_action_delay=MAX_ACTION_DELAY,
    fixed_durability=FIXED_DURABILITY,
    continuous_actions=True,
)

print(f"SAC Performance (500 episodes, original metric):")
print(f"  Mean reward: {np.mean(sac_rewards):.2f}")
print(f"  Std reward:  {np.std(sac_rewards):.2f}")
print(f"  Min reward:  {np.min(sac_rewards):.2f}")
print(f"  Max reward:  {np.max(sac_rewards):.2f}")

In [ ]:
# Compare SAC vs baselines
fig, ax = plt.subplots(figsize=(10, 5))

all_results = {name: stats['mean'] for name, stats in results.items()}
all_results['SAC'] = np.mean(sac_rewards)

sorted_results = sorted(all_results.items(), key=lambda x: x[1])
names = [r[0] for r in sorted_results]
means = [r[1] for r in sorted_results]

colors = ['green' if n == 'SAC' else 'steelblue' for n in names]
bars = ax.barh(names, means, color=colors)

ax.set_xlabel('Mean Reward')
ax.set_title('SAC vs Baseline Policies')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

for bar, mean in zip(bars, means):
    ax.text(mean + 5, bar.get_y() + bar.get_height()/2, 
            f'{mean:.1f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 5. Analyse SAC Behaviour

In [ ]:
# Analyse action (delay) distribution across different states
import pandas as pd

env = ServiceEnv(
    scenario, max_time=MAX_TIME, seed=42,
    max_action_delay=MAX_ACTION_DELAY,
    continuous_actions=True,
)

state_actions = []
for ep in range(100):
    obs, _ = env.reset(seed=42 + ep)
    done = False
    while not done:
        action, _ = trained_model.predict(obs, deterministic=True)
        delay = float(action[0])  # Continuous action is the delay directly
        state_actions.append({
            'current_time': obs[0] * MAX_TIME,
            'last_interval': obs[1] * MAX_TIME,
            'service_count': obs[2] * 100,
            'avg_interval': obs[3] * MAX_TIME,
            'durability': obs[4] * 10,
            'delay': delay
        })
        obs, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

df = pd.DataFrame(state_actions)

print("Delay statistics:")
print(f"  Mean: {df['delay'].mean():.1f}")
print(f"  Std:  {df['delay'].std():.1f}")
print(f"  Min:  {df['delay'].min():.1f}")
print(f"  Max:  {df['delay'].max():.1f}")
print(f"\nTotal decisions: {len(df)}")

# Optimal: 24.2 + 24.2 * 1.0 = 48.4 for durability=1.0
print(f"\nOptimal delay for durability=1.0: 48.4")

In [ ]:
# Plot delay distribution and relationships
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Histogram of delays
ax = axes[0]
ax.hist(df['delay'], bins=30, edgecolor='black', alpha=0.7)
ax.axvline(48.4, color='red', linestyle='--', label='Optimal (48.4)')
ax.set_xlabel('Delay')
ax.set_ylabel('Count')
ax.set_title('Distribution of Chosen Delays')
ax.legend()

# Delay vs durability
ax = axes[1]
ax.scatter(df['durability'], df['delay'], alpha=0.5, s=20)
# Plot optimal linear policy for reference
d_range = np.linspace(df['durability'].min(), df['durability'].max(), 50)
ax.plot(d_range, 24.2 + 24.2 * d_range, 'r--', label='Optimal linear')
ax.set_xlabel('Durability')
ax.set_ylabel('Chosen Delay')
ax.set_title('Delay vs Durability')
ax.legend()

# Delay vs time
ax = axes[2]
scatter = ax.scatter(df['current_time'], df['delay'], c=df['durability'], alpha=0.5, s=20, cmap='viridis')
ax.set_xlabel('Current Time')
ax.set_ylabel('Chosen Delay')
ax.set_title('Delay vs Time (coloured by durability)')
plt.colorbar(scatter, ax=ax, label='Durability')

plt.tight_layout()
plt.show()

## 6. Save Model

In [ ]:
# Save trained model
model_path = '../models/sac_service_policy'
trained_model.save(model_path)
print(f"Model saved to {model_path}")